In [1]:
!pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 660.6 kB/s eta 0:00:02
   ---- ----------------------------------- 0.1/1.2 MB 1.2 MB/s eta 0:00:01
   -------- ------------------------------- 0.2/1.2 MB 1.6 MB/s eta 0:00:01
   ----------- ---------------------------- 0.3/1.2 MB 1.8 MB/s eta 0:00:01
   -------------- ------------------------- 0.4/1.2 MB 1.8 MB/s eta 0:00:01
   --------------- ------------------------ 0.5/1.2 MB 1.6 MB/s eta 0:00:01
   ---------------- ----------------------- 0.5/1.2 MB 1.5 MB/s eta 0:00:01
   ----------------- ---------------------- 0.5/1.2 MB 1.4 MB/s eta 0:00:01
   ------------------- -------------------- 0.6/1.2 MB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 0.6/1.2 MB 1.3 MB/s eta 0:00:01
   -------------------- ------------------- 0.6/1.2 MB 1.2 MB/s eta 0:00:01
   ---------------------

In [36]:
!pip install pgvector

## Create the PGVector database

The goal here is to make a Postgresql relational database and populate it with sentence embeddings. I use PGVector and psycopg2 to do this. The embeddings are used in the chatbot via similarity search to add context to the prompt, a technique called "retrieval augmented generation".

In [43]:
import json
import psycopg2
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import PGVector
from langchain.schema import Document

In [44]:
# define an embedding model using a pretrained model from HuggingFace
embedding_model_id = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
embedding_model = HuggingFaceEmbeddings(
  model_name=embedding_model_id,
  model_kwargs=model_kwargs,
  encode_kwargs=encode_kwargs,
)

c:\Users\szjia\Documents\WaM\Spring 2024\DATA 340 NLP\Final Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [46]:
# Set up the connection between PGVector and Postgresql, 
# note that database st_line has to be set up beforehand through Postgresql.
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver="psycopg2",
    host="localhost",
    port=5432,
    database="st_line",
    user="postgres",
    password="password",
)

COLLECTION_NAME = "character_quotes"
store = PGVector(
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding_function=embedding_model
)


In [47]:
# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="st_line",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

# Read JSON file
with open('C:/Users/szjia/Documents/WaM/Spring 2024/DATA 340 NLP/Final Project/line_data/lines_by_character.json', 'r', encoding="utf-8") as f:
    data = json.load(f)

# Loop through the data which is the in format of character : [lines] in a key-value pair.
id = 0
for character in data:
    character_name = character
    for quote in data[character]:
      quote_text = quote
      # print(character_name + ": " + quote)
      # use the PGVector connection to encode all character lines and add them into the database as a table.
      store.add_documents(
          [Document(page_content=quote_text, metadata={"document_id": id, "author": character})]
      )
      id += 1



## Bug Testing Code

If you are trying to populate your own pgvector database, you do not need to run the code below.

In [16]:
conn = psycopg2.connect(
    dbname="st_line",
    user="postgres",
    password="iusegmail88",
    host="localhost",
    port="5432"
)
cur = conn.cursor()
# cur.execute("SELECT * FROM character_quotes WHERE character_name IS Picard;")
# cur.close()

In [22]:
cur.execute("SELECT DISTINCT character_name FROM character_quotes;")
result = cur.fetchall()
print(result)

[('NURIA',), ('HOLO-PICARD',), ('CORIN',), ('MARTHA',), ('MONROE',), ('REED',), ('BEGGAR',), ('KAJADA',), ('KYLE',), ('TALAXIAN',), ('WYKOFF',), ('RABBIT',), ('GAAVRIN',), ('FISHER',), ('GORGON',), ('MORIN',), ('JUNIOR',), ('KANE',), ('GIRL ON BLADES',), ('MULLIBOK',), ('JIMMY',), ('RODRIGUEZ',), ('KRYTON',), ('SKALARR',), ('LINKE',), ('DARA',), ('GIOTTO',), ('LEUCON',), ('PTERA',), ('KARNAS',), ('KAPLAN',), ('IRUDAN',), ('MAKLII',), ('ATTACKER',), ('LAUNCH CONTROL',), ('TAIN',), ('DATHON',), ('NEWSCASTER',), ('NAHSK',), ('KORINAS',), ('SHILAT',), ('DROXINE',), ('DECIUS',), ('TREVIS',), ('QUAICE',), ('JORDAN',), ('RONDON',), ('JOHNNY',), ('MIDRO',), ('GREER',), ('CREW',), ('RYAN',), ('KIPP',), ('MANTOOS',), ('VAREL',), ("TRENA'L",), ('ALICE',), ('BRENT',), ("T'SU",), ('ETHAN',), ('ENINA',), ('KARA',), ('FARRELL',), ('ARRIDOR',), ('SANDAL MAKER',), ('BROSSMER',), ('MEA',), ('JEV',), ('BARMAN',), ("IXTANA'RAX",), ('KINGSLEY',), ('ELDER',), ('MUROC',), ('BENZAN',), ('DORAN',), ('TOLAR',),

In [23]:
cur.execute("SELECT * FROM character_quotes WHERE character_name='PICARD';")
result = cur.fetchall()
print(result)

[(47398, 'PICARD', "That'll have to wait."), (47555, 'PICARD', 'Destination?'), (47854, 'PICARD', 'Bags of mostly water?'), (49651, 'PICARD', 'Surrounding the structure.'), (49731, 'PICARD', 'Not something I do easily.'), (46413, 'PICARD', 'I see no charges against us, Your Honour.'), (46414, 'PICARD', 'Guilty. Provisionally.'), (46485, 'PICARD', 'Tune that down! Commander Riker, come in. Can you hear me?'), (46360, 'PICARD', "You will agree, Data, that Starfleet's orders are difficult?"), (46361, 'PICARD', 'As simple as that.'), (46362, 'PICARD', "It's hardly simple, Data, to negotiate a friendly agreement for Starfleet to use the base while at the same time snoop around finding how and why the life form there built it."), (46363, 'PICARD', 'Data, how can you be programmed as a virtual encyclopedia of human information without knowing a simple word like snoop?'), (46364, 'PICARD', 'It means to spy, to sneak.'), (46365, 'PICARD', 'Exactly, yes.'), (46366, 'PICARD', 'Shut off that damne

In [24]:
# Close the cursor and connection
cur.close()
conn.close()